In [4]:
# One-step automatic finder + cleaner for Colab (copy & paste into a new cell)
# -- mounts Drive, finds your notebook by name pattern, cleans widget metadata, saves cleaned copy

# Install nbformat if needed
!pip install -q nbformat

import os, nbformat, glob, shutil
from google.colab import drive

# 1) Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# 2) Search Drive for likely notebook files (change pattern if needed)
search_root = "/content/drive/MyDrive"
pattern = "**/*Real*Time*.ipynb"   # matches filenames with Real and Time (covers variants)
print("Searching Drive under:", search_root)
matches = glob.glob(os.path.join(search_root, pattern), recursive=True)

if not matches:
    # also check top-level /content in case you uploaded directly to Colab session
    print("No matches in Drive. Searching /content/ for uploaded notebooks...")
    matches = glob.glob("/content/*Real*Time*.ipynb") + glob.glob("/content/*Real*Time*.ipynb/**")

if not matches:
    print("No notebook found with pattern. List top-level files in /content and in Drive root:")
    print("\n/content files:")
    print(os.listdir("/content")[:50])
    print("\nDrive root files (MyDrive):")
    print(os.listdir(search_root)[:50])
    raise FileNotFoundError("No Real-Time notebook found. Please upload or confirm the filename and try again.")

# Show matches and pick first (you can change index if multiple)
print("\nFound notebook candidates:")
for i, p in enumerate(matches):
    try:
        size_mb = os.path.getsize(p) / (1024*1024)
    except:
        size_mb = 0
    print(f"{i}: {p}  ({size_mb:.2f} MB)")

idx = 0   # change this index to pick a different match if multiple
input_file = matches[idx]
print("\nWill clean this file:", input_file)

# 3) Read notebook
nb = nbformat.read(input_file, as_version=4)

# 4) Remove widget/extension metadata from each cell
removed_count = 0
for cell in nb.cells:
    meta = cell.get("metadata", {})
    if "widgets" in meta:
        del meta["widgets"]
        removed_count += 1
    if "extensions" in meta:
        meta.pop("extensions", None)
        removed_count += 1

print(f"Removed widget/extension metadata entries in {removed_count} cells (if present).")

# 5) Save cleaned notebook to /content and also copy to Drive
output_local = "/content/cleaned_notebook.ipynb"
nbformat.write(nb, output_local)
print("Saved cleaned notebook to:", output_local)

# Also copy to Drive so you can download from Drive GUI
drive_out = os.path.join(search_root, "cleaned_notebook.ipynb")
shutil.copy(output_local, drive_out)
print("Also copied cleaned notebook to your Drive at:", drive_out)

print("\n✅ Cleaning complete. Download the cleaned notebook from Drive or from /content and upload THAT to GitHub.")


Mounted at /content/drive
Searching Drive under: /content/drive/MyDrive

Found notebook candidates:
0: /content/drive/MyDrive/Colab Notebooks/Real-Time NLP Sentiment Analysis.ipynb  (1.32 MB)
1: /content/drive/MyDrive/Colab Notebooks/Copy of Real-Time NLP Sentiment Analysis.ipynb  (1.32 MB)

Will clean this file: /content/drive/MyDrive/Colab Notebooks/Real-Time NLP Sentiment Analysis.ipynb
Removed widget/extension metadata entries in 0 cells (if present).
Saved cleaned notebook to: /content/cleaned_notebook.ipynb
Also copied cleaned notebook to your Drive at: /content/drive/MyDrive/cleaned_notebook.ipynb

✅ Cleaning complete. Download the cleaned notebook from Drive or from /content and upload THAT to GitHub.
